# Predicting AFL winners using Machine Learning - Cleaning & Feature Preprocessing

---

<img src='img/sportsbet.png' 
style="height:200px;width:700px;">

---



Using the data provided by Sportsbet and CIKM 2015 challenge. We are going to try and maximize winnnings from bet tippings in any given season of AFL. This involves two steps:

* Part 1: Predicting with a high enough accuracy the probability of the home team winning. 

* Part 2: Calculating what the best percentage of our capital to bet is.



The data for this project is sourced from several websites. The meat of the dataset is provided under the **2015 CIKM & SportsBet AFL Challenge**, with others sections such as historical ladder rankings and historical betting odds coming from 

> [The Official AFL Site](http://www.afl.com.au/ladder)
<img src='img/ladder.png'>

---

> [AFL Tables](https://afltables.com/afl/stats/stats_idx.html)
<img src='img/afltables.png'>

---

>[aussportsbetting.com](http://www.aussportsbetting.com/data/historical-afl-results-and-odds-data/)
<img src='img/aussportsbetting.png'>

Since we are unable to use the game statistics of the current game to predict the outcome ("Duh! Otherwise we'd all be millionaires.") We are going to have to do a lot of feature preprocessing to create predictors for our models. Some important ones that come to mind are:

* Previous year's ladder score

* Head-on-head data

* Current form for the team

* Premiership match?

But first lets import some dependencies

#### Import Dependencies
---

In [103]:
# Imports dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

%matplotlib inline

In [104]:
data = pd.read_csv("data/afl_2017_test.csv")

In [105]:
data["Date"] = pd.to_datetime(data["Date"])

In [106]:
data["Date"][0].year

2018

In [107]:
data["season"] = data["Date"].map(lambda x: x.year)

In [108]:
data.rename(columns={'Home Team': 'h_team', 'Away Team': 'a_team', "Play Off Game?": "play_off",
                     "Home Odds": "home_odds", "Away Odds": "away_odds"}, inplace=True)

In [109]:
data.h_team.unique()

array(['West Coast', 'Melbourne', 'GWS Giants', 'Hawthorn', 'Gold Coast',
       'Port Adelaide', 'St Kilda', 'Essendon', 'Richmond', 'Adelaide',
       'Geelong', 'Sydney', 'Brisbane', 'Collingwood', 'Fremantle',
       'Carlton', 'Western Bulldogs', 'North Melbourne'], dtype=object)

#### Team Names and Season Done
---

In [110]:
data["play_off"] = data["play_off"].map(lambda x: 1 if x == "Y" else 0)

In [111]:
data.drop(["Home Goals", "Home Behinds", "Away Goals", "Away Behinds", "Venue", "Kick Off (local)"], inplace = True, axis = 1)

In [112]:
data["margin"] = data["Home Score"] - data["Away Score"]

In [113]:
data.head()

,Date,h_team,a_team,Home Score,Away Score,play_off,home_odds,away_odds,season,margin
0,2018-03-25,West Coast,Sydney,0,0,0,2.60,1.50,2018,0
1,2018-03-25,Melbourne,Geelong,0,0,0,2.35,1.60,2018,0
2,2018-03-25,GWS Giants,Western Bulldogs,0,0,0,1.45,2.75,2018,0
3,2018-03-24,Hawthorn,Collingwood,0,0,0,1.95,1.85,2018,0
4,2018-03-24,Gold Coast,North Melbourne,0,0,0,1.85,1.95,2018,0


#### Playoff games done
---

In [114]:
# Loads the ladder rankings: ladder_rankings
ladder_rankings = pd.read_csv("data/ladder_rankings.csv", index_col = 1)

In [115]:
ladder_rankings.head()

,tid,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007,2006,2005,2004,2003,2002,2001
tname,,,,,,,,,,,,,,,,,,
Adelaide,101.0,62.0,64.0,54.0,44.0,40.0,68.0,28.0,36.0,56.0,52.0,48.0,64.0,68.0,68.0,14.0,60.0,48.0
Brisbane,102.0,20.0,12.0,16.0,28.0,40.0,40.0,16.0,28.0,54.0,40.0,40.0,28.0,40.0,64.0,16.0,68.0,68.0
Carlton,103.0,24.0,28.0,16.0,30.0,44.0,44.0,58.0,44.0,52.0,40.0,16.0,14.0,18.0,64.0,20.0,12.0,56.0
Collingwood,104.0,38.0,36.0,40.0,44.0,56.0,64.0,80.0,70.0,60.0,48.0,52.0,56.0,20.0,60.0,28.0,52.0,44.0
Essendon,105.0,48.0,12.0,24.0,50.0,56.0,44.0,46.0,28.0,42.0,32.0,40.0,14.0,32.0,56.0,30.0,50.0,68.0


In [116]:
def get_prev_score(tname, year):
    """
    Returns the previous years ladder scores given home and away team id's and the year
    """
    try:
        prev_score = ladder_rankings.loc[tname, str(year-1)]
    except KeyError:
        prev_score = 0     
    
    return prev_score

In [117]:
# Initializes and concats the previous year scores for the home and away team: 'h_prev_score', 'a_prev_score'
data['h_prevladder_score'] = data.apply(lambda row: get_prev_score(row["h_team"], row["season"]), axis=1)
data['a_prevladder_score'] = data.apply(lambda row: get_prev_score(row["a_team"], row["season"]), axis=1)

#### Previous Ladder score done
---

In [118]:
data.head(5)

,Date,h_team,a_team,Home Score,Away Score,play_off,home_odds,away_odds,season,margin,h_prevladder_score,a_prevladder_score
0,2018-03-25,West Coast,Sydney,0,0,0,2.60,1.50,2018,0,48.0,56.0
1,2018-03-25,Melbourne,Geelong,0,0,0,2.35,1.60,2018,0,48.0,62.0
2,2018-03-25,GWS Giants,Western Bulldogs,0,0,0,1.45,2.75,2018,0,60.0,44.0
3,2018-03-24,Hawthorn,Collingwood,0,0,0,1.95,1.85,2018,0,42.0,38.0
4,2018-03-24,Gold Coast,North Melbourne,0,0,0,1.85,1.95,2018,0,24.0,24.0


In [119]:
def get_previous_margins(home_team, away_team, date):
    """
    Obtains the margin for the previous X games (5 being default) between these two teams
    
    """
    
    years_to_shift = 5
    
    # Sets the condition such that either team can be home or away: condition
    condition = ((data["a_team"]==home_team) & (data["h_team"]==away_team))|\
                ((data["h_team"]==home_team) & (data["a_team"]==away_team))
    
    # Finds all games in the full_matches dataframe that fits this condition: previous_faceoffs
    previous_faceoffs = data[condition]
    
    # Initiatlizes the index for the current match: index
    index = previous_faceoffs.index[previous_faceoffs["Date"] == date][0]
    
    # Changes the margin to a negative if the home team in the current game, plays as an away team in a previous game
    previous_faceoffs["margin"] = previous_faceoffs.apply(lambda row: -row["margin"] if home_team != row["h_team"]
                                            else row["margin"], axis=1)
    
    # Shifts the data depending on years_to_shift and concats it among itself: margins_data
    margins_data = pd.concat((previous_faceoffs[["margin"]].shift(-x).add_suffix("_year{}".format(x)) for x in range(1,years_to_shift+1)), axis = 1)
    
    print(margins_data)
    
    # Concats the data with the previous faceoffs dataframe: combined_data
    combined_data = pd.concat([previous_faceoffs, margins_data], axis = 1)
    
    # Returns the margins of the previous X games
    return combined_data.loc[index,"margin_year1":]

In [120]:
data.h_team.unique()

array(['West Coast', 'Melbourne', 'GWS Giants', 'Hawthorn', 'Gold Coast',
       'Port Adelaide', 'St Kilda', 'Essendon', 'Richmond', 'Adelaide',
       'Geelong', 'Sydney', 'Brisbane', 'Collingwood', 'Fremantle',
       'Carlton', 'Western Bulldogs', 'North Melbourne'], dtype=object)

In [121]:
# Initiates 5 columns filled with 0s to be filled with previous margins data
data["margins_prev_match1"] = 0
data["margins_prev_match2"] = 0
data["margins_prev_match3"] = 0
data["margins_prev_match4"] = 0
data["margins_prev_match5"] = 0

data.head(3)

,Date,h_team,a_team,Home Score,Away Score,play_off,home_odds,away_odds,season,margin,h_prevladder_score,a_prevladder_score,margins_prev_match1,margins_prev_match2,margins_prev_match3,margins_prev_match4,margins_prev_match5
0,2018-03-25,West Coast,Sydney,0,0,0,2.60,1.50,2018,0,48.0,56.0,0,0,0,0,0
1,2018-03-25,Melbourne,Geelong,0,0,0,2.35,1.60,2018,0,48.0,62.0,0,0,0,0,0
2,2018-03-25,GWS Giants,Western Bulldogs,0,0,0,1.45,2.75,2018,0,60.0,44.0,0,0,0,0,0


In [122]:
# Applies a lambda function which makes the turns the margin into a negative if winning team is not the same the home team

for index, row in data.iterrows():
    margins = get_previous_margins(row["h_team"], row["a_team"], row["Date"]);
    data.iloc[index,-5:] = list(margins);

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
0             26.0         -39.0          52.0         -28.0         -34.0
188          -39.0          52.0         -28.0         -34.0         -52.0
385           52.0         -28.0         -34.0         -52.0         -13.0
486          -28.0         -34.0         -52.0         -13.0         -52.0
702          -34.0         -52.0         -13.0         -52.0           NaN
899          -52.0         -13.0         -52.0           NaN           NaN
1115         -13.0         -52.0           NaN           NaN           NaN
1426         -52.0           NaN           NaN           NaN           NaN
1598           NaN           NaN           NaN           NaN           NaN
      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
1            -29.0        -111.0          24.0         -66.0         -68.0
194         -111.0          24.0         -66.0         -68.0         -43.0
232           24.0       

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
13           -46.0         -37.0         -38.0          32.0         -43.0
53           -37.0         -38.0          32.0         -43.0        -110.0
218          -38.0          32.0         -43.0        -110.0          44.0
295           32.0         -43.0        -110.0          44.0          21.0
472          -43.0        -110.0          44.0          21.0          34.0
570         -110.0          44.0          21.0          34.0          -6.0
754           44.0          21.0          34.0          -6.0         -13.0
859           21.0          34.0          -6.0         -13.0          27.0
1015          34.0          -6.0         -13.0          27.0          53.0
1059          -6.0         -13.0          27.0          53.0          67.0
1147         -13.0          27.0          53.0          67.0           5.0
1272          27.0          53.0          67.0           5.0           NaN
1418          53.0       

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
25            -4.0         -46.0         -35.0         -37.0          25.0
114          -46.0         -35.0         -37.0          25.0          33.0
315          -35.0         -37.0          25.0          33.0          83.0
388          -37.0          25.0          33.0          83.0          42.0
483           25.0          33.0          83.0          42.0          88.0
539           33.0          83.0          42.0          88.0           0.0
737           83.0          42.0          88.0           0.0           1.0
947           42.0          88.0           0.0           1.0           NaN
1154          88.0           0.0           1.0           NaN           NaN
1353           0.0           1.0           NaN           NaN           NaN
1536           1.0           NaN           NaN           NaN           NaN
1622           NaN           NaN           NaN           NaN           NaN
      margin_year1  margi

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
39             7.0          69.0          -3.0         -24.0          10.0
281           69.0          -3.0         -24.0          10.0           3.0
619           -3.0         -24.0          10.0           3.0          36.0
786          -24.0          10.0           3.0          36.0         -26.0
1011          10.0           3.0          36.0         -26.0         -29.0
1153           3.0          36.0         -26.0         -29.0           NaN
1253          36.0         -26.0         -29.0           NaN           NaN
1338         -26.0         -29.0           NaN           NaN           NaN
1500         -29.0           NaN           NaN           NaN           NaN
1553           NaN           NaN           NaN           NaN           NaN
      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
7            -43.0         -65.0         -82.0        -112.0           9.0
40           -65.0       

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
13           -46.0         -37.0         -38.0          32.0         -43.0
53           -37.0         -38.0          32.0         -43.0        -110.0
218          -38.0          32.0         -43.0        -110.0          44.0
295           32.0         -43.0        -110.0          44.0          21.0
472          -43.0        -110.0          44.0          21.0          34.0
570         -110.0          44.0          21.0          34.0          -6.0
754           44.0          21.0          34.0          -6.0         -13.0
859           21.0          34.0          -6.0         -13.0          27.0
1015          34.0          -6.0         -13.0          27.0          53.0
1059          -6.0         -13.0          27.0          53.0          67.0
1147         -13.0          27.0          53.0          67.0           5.0
1272          27.0          53.0          67.0           5.0           NaN
1418          53.0       

...

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
107          -24.0           3.0          74.0          29.0          12.0
204            3.0          74.0          29.0          12.0          11.0
386           74.0          29.0          12.0          11.0           5.0
427           29.0          12.0          11.0           5.0          56.0
532           12.0          11.0           5.0          56.0         -20.0
700           11.0           5.0          56.0         -20.0          47.0
994            5.0          56.0         -20.0          47.0         -27.0
1044          56.0         -20.0          47.0         -27.0           NaN
1224         -20.0          47.0         -27.0           NaN           NaN
1442          47.0         -27.0           NaN           NaN           NaN
1542         -27.0           NaN           NaN           NaN           NaN
1664           NaN           NaN           NaN           NaN           NaN
      margin_year1  margi

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
67            50.0          70.0          97.0          71.0          59.0
142           70.0          97.0          71.0          59.0          16.0
247           97.0          71.0          59.0          16.0          29.0
442           71.0          59.0          16.0          29.0         -28.0
663           59.0          16.0          29.0         -28.0          25.0
864           16.0          29.0         -28.0          25.0          15.0
1005          29.0         -28.0          25.0          15.0          -8.0
1108         -28.0          25.0          15.0          -8.0          -1.0
1174          25.0          15.0          -8.0          -1.0           NaN
1256          15.0          -8.0          -1.0           NaN           NaN
1277          -8.0          -1.0           NaN           NaN           NaN
1628          -1.0           NaN           NaN           NaN           NaN
1677           NaN       

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
119          -88.0         -46.0         -79.0         -86.0         -40.0
324          -46.0         -79.0         -86.0         -40.0          -4.0
573          -79.0         -86.0         -40.0          -4.0         103.0
639          -86.0         -40.0          -4.0         103.0         -19.0
801          -40.0          -4.0         103.0         -19.0         -28.0
971           -4.0         103.0         -19.0         -28.0          47.0
1152         103.0         -19.0         -28.0          47.0          57.0
1313         -19.0         -28.0          47.0          57.0           NaN
1403         -28.0          47.0          57.0           NaN           NaN
1460          47.0          57.0           NaN           NaN           NaN
1555          57.0           NaN           NaN           NaN           NaN
1689           NaN           NaN           NaN           NaN           NaN
      margin_year1  margi

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
84           -60.0         -69.0         -56.0         -62.0         -25.0
235          -69.0         -56.0         -62.0         -25.0          -1.0
396          -56.0         -62.0         -25.0          -1.0           5.0
484          -62.0         -25.0          -1.0           5.0         -38.0
641          -25.0          -1.0           5.0         -38.0         -29.0
820           -1.0           5.0         -38.0         -29.0         -63.0
851            5.0         -38.0         -29.0         -63.0         -81.0
935          -38.0         -29.0         -63.0         -81.0          43.0
1208         -29.0         -63.0         -81.0          43.0           NaN
1315         -63.0         -81.0          43.0           NaN           NaN
1501         -81.0          43.0           NaN           NaN           NaN
1572          43.0           NaN           NaN           NaN           NaN
1701           NaN       

      margin_year1  margin_year2  margin_year3  margin_year4  margin_year5
19            67.0           9.0         -16.0         -26.0         -44.0
85             9.0         -16.0         -26.0         -44.0         -64.0
240          -16.0         -26.0         -44.0         -64.0         -17.0
495          -26.0         -44.0         -64.0         -17.0          -8.0
647          -44.0         -64.0         -17.0          -8.0           0.0
716          -64.0         -17.0          -8.0           0.0          21.0
926          -17.0          -8.0           0.0          21.0          38.0
1033          -8.0           0.0          21.0          38.0          56.0
1168           0.0          21.0          38.0          56.0           NaN
1437          21.0          38.0          56.0           NaN           NaN
1469          38.0          56.0           NaN           NaN           NaN
1551          56.0           NaN           NaN           NaN           NaN
1713           NaN       

In [123]:
data.head()

,Date,h_team,a_team,Home Score,Away Score,play_off,home_odds,away_odds,season,margin,h_prevladder_score,a_prevladder_score,margins_prev_match1,margins_prev_match2,margins_prev_match3,margins_prev_match4,margins_prev_match5
0,2018-03-25,West Coast,Sydney,0,0,0,2.60,1.50,2018,0,48.0,56.0,26.0,-39.0,52.0,-28.0,-34.0
1,2018-03-25,Melbourne,Geelong,0,0,0,2.35,1.60,2018,0,48.0,62.0,-29.0,-111.0,24.0,-66.0,-68.0
2,2018-03-25,GWS Giants,Western Bulldogs,0,0,0,1.45,2.75,2018,0,60.0,44.0,48.0,2.0,-6.0,25.0,-45.0
3,2018-03-24,Hawthorn,Collingwood,0,0,0,1.95,1.85,2018,0,42.0,38.0,24.0,-18.0,1.0,10.0,65.0
4,2018-03-24,Gold Coast,North Melbourne,0,0,0,1.85,1.95,2018,0,24.0,24.0,19.0,-13.0,-38.0,55.0,43.0


In [124]:
data[data["season"] == 2009]

,Date,h_team,a_team,Home Score,Away Score,play_off,home_odds,away_odds,season,margin,h_prevladder_score,a_prevladder_score,margins_prev_match1,margins_prev_match2,margins_prev_match3,margins_prev_match4,margins_prev_match5
1632,2009-09-26,St Kilda,Geelong,68,80,0,2.20,1.63,2009,-12,52.0,84.0,6.0,NaN,NaN,NaN,NaN
1633,2009-09-19,Geelong,Collingwood,120,47,0,1.34,3.18,2009,73,84.0,48.0,NaN,NaN,NaN,NaN,NaN
1634,2009-09-18,St Kilda,Western Bulldogs,60,53,0,1.36,3.05,2009,7,52.0,62.0,45.0,NaN,NaN,NaN,NaN
1635,2009-09-12,Collingwood,Adelaide,83,78,0,2.18,1.66,2009,5,48.0,52.0,21.0,NaN,NaN,NaN,NaN
1636,2009-09-11,Western Bulldogs,Brisbane,107,56,0,1.26,3.70,2009,51,62.0,40.0,18.0,NaN,NaN,NaN,NaN
1637,2009-09-06,St Kilda,Collingwood,80,52,0,1.60,2.25,2009,28,52.0,48.0,NaN,NaN,NaN,NaN,NaN
1638,2009-09-05,Brisbane,Carlton,111,104,0,1.87,1.86,2009,7,40.0,40.0,NaN,NaN,NaN,NaN,NaN
1639,2009-09-05,Geelong,Western Bulldogs,96,82,0,1.83,1.93,2009,14,84.0,62.0,-14.0,NaN,NaN,NaN,NaN
1640,2009-09-04,Adelaide,Essendon,166,70,0,1.10,6.50,2009,96,52.0,32.0,NaN,NaN,NaN,NaN,NaN
1641,2009-08-30,Collingwood,Western Bulldogs,76,100,0,1.71,2.08,2009,-24,48.0,62.0,1.0,NaN,NaN,NaN,NaN


In [125]:
data["win_loc"] = data.apply(lambda row: 1 if row["Home Score"] > row["Away Score"] else 0, axis = 1)

In [126]:
data.isnull().sum()

Date                     0
h_team                   0
a_team                   0
Home Score               0
Away Score               0
play_off                 0
home_odds                0
away_odds                0
season                   0
margin                   0
h_prevladder_score       0
a_prevladder_score       0
margins_prev_match1    153
margins_prev_match2    306
margins_prev_match3    459
margins_prev_match4    612
margins_prev_match5    765
win_loc                  0
dtype: int64

In [132]:
data.dropna(inplace=True)

In [133]:
data.to_csv("data/afl_2018_test.csv")